In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_3/models/B5R2b5_5FP_1FC_fold3_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold3.csv') #Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4598, 25)
(4598, 25)
Normal:  (3003, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4598 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.45463496,  0.38546577, -0.18478562, ...,  0.05667846,
         0.28048524,  0.84149384],
       [-0.03259616,  0.27331766, -0.17444794, ..., -0.10579466,
        -0.14910072, -0.02099088],
       [-0.13000685, -0.17665119, -0.16754684, ..., -0.06661568,
        -0.22860941,  0.29183546],
       ...,
       [-0.18753202, -0.17503873,  0.56931275, ...,  0.36215997,
         0.3850328 , -0.07924222],
       [-0.19728118, -0.20151325,  0.53080094, ...,  0.31086406,
         0.38414007,  0.49989733],
       [-0.19122176, -0.18163368, -0.23568964, ...,  0.84365398,
        -0.2383945 ,  0.97001588]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.454635,0.385466,-0.184786,-0.028337,-0.053728,-0.153055,-0.194530,0.648894,0.435559,0.533567,...,-0.027883,-0.127334,-0.042394,0.394820,0.066778,1.025149,0.743229,0.056678,0.280485,0.841494
1,-0.032596,0.273318,-0.174448,0.491713,0.028845,0.026885,0.706252,0.480919,-0.003031,0.310332,...,-0.089946,-0.187004,-0.214560,-0.148369,-0.155087,0.280151,0.304668,-0.105795,-0.149101,-0.020991
2,-0.130007,-0.176651,-0.167547,0.592732,0.117312,-0.016051,0.808872,0.016193,-0.181890,-0.048170,...,-0.154560,-0.174229,-0.228628,-0.167602,-0.174331,0.012112,-0.003200,-0.066616,-0.228609,0.291835
3,-0.016664,-0.189828,-0.199131,-0.174706,-0.138387,-0.204502,-0.210086,0.073383,-0.106316,0.100140,...,0.071982,-0.074183,-0.105384,0.376861,-0.211068,0.351206,0.207486,-0.132347,0.176357,0.114247
4,0.777053,0.669774,-0.181162,-0.017197,-0.140167,-0.047103,-0.187592,0.614855,0.332440,0.779578,...,0.509492,-0.115893,-0.085294,0.647234,-0.201471,1.124767,0.949034,-0.155360,0.101219,0.577824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4593,0.428170,-0.148052,-0.209006,-0.062886,-0.161224,-0.211183,-0.203773,0.391663,-0.003610,0.546784,...,0.409239,-0.150543,0.081394,0.110912,-0.189382,0.632477,0.314648,-0.166028,-0.153261,-0.050434
4594,0.277539,0.267934,-0.082643,-0.114626,-0.148996,0.051271,-0.164587,0.386695,0.030218,0.591794,...,0.457947,-0.115593,0.064719,0.501672,-0.174160,0.392934,0.944996,-0.093637,-0.022131,0.527060
4595,-0.187532,-0.175039,0.569313,0.093905,0.506204,-0.105238,0.091066,-0.173115,0.134328,-0.229321,...,-0.015995,0.674306,0.395522,-0.023896,0.409509,-0.198020,-0.203667,0.362160,0.385033,-0.079242
4596,-0.197281,-0.201513,0.530801,0.517311,0.690652,-0.194849,0.121112,-0.151347,0.140172,-0.195687,...,-0.102787,0.559331,0.471969,-0.050557,0.754076,-0.171817,-0.119111,0.310864,0.384140,0.499897


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.454635,0.385466,-0.184786,-0.028337,-0.053728,-0.153055,-0.194530,0.648894,0.435559,0.533567,...,0.066778,1.025149,0.743229,0.056678,0.280485,0.841494,Abnormal,P31,FP-B,AB01
1,-0.032596,0.273318,-0.174448,0.491713,0.028845,0.026885,0.706252,0.480919,-0.003031,0.310332,...,-0.155087,0.280151,0.304668,-0.105795,-0.149101,-0.020991,Abnormal,P1,FP-A,AB01
2,-0.130007,-0.176651,-0.167547,0.592732,0.117312,-0.016051,0.808872,0.016193,-0.181890,-0.048170,...,-0.174331,0.012112,-0.003200,-0.066616,-0.228609,0.291835,Abnormal,P2,FP-A,AB01
3,-0.016664,-0.189828,-0.199131,-0.174706,-0.138387,-0.204502,-0.210086,0.073383,-0.106316,0.100140,...,-0.211068,0.351206,0.207486,-0.132347,0.176357,0.114247,Abnormal,P42,FP-B,AB01
4,0.777053,0.669774,-0.181162,-0.017197,-0.140167,-0.047103,-0.187592,0.614855,0.332440,0.779578,...,-0.201471,1.124767,0.949034,-0.155360,0.101219,0.577824,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4593,0.428170,-0.148052,-0.209006,-0.062886,-0.161224,-0.211183,-0.203773,0.391663,-0.003610,0.546784,...,-0.189382,0.632477,0.314648,-0.166028,-0.153261,-0.050434,Normal,P32,FP-B,Normal
4594,0.277539,0.267934,-0.082643,-0.114626,-0.148996,0.051271,-0.164587,0.386695,0.030218,0.591794,...,-0.174160,0.392934,0.944996,-0.093637,-0.022131,0.527060,Normal,P42,FP-B,Normal
4595,-0.187532,-0.175039,0.569313,0.093905,0.506204,-0.105238,0.091066,-0.173115,0.134328,-0.229321,...,0.409509,-0.198020,-0.203667,0.362160,0.385033,-0.079242,Normal,P52,FP-C,Normal
4596,-0.197281,-0.201513,0.530801,0.517311,0.690652,-0.194849,0.121112,-0.151347,0.140172,-0.195687,...,0.754076,-0.171817,-0.119111,0.310864,0.384140,0.499897,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_5FP_RF_5FP_fold3_1.csv') #เปลี่ยนชื่อไฟล์